In [22]:
!pip install kagglehub

import pandas as pd
import kagglehub

# Download dataset
path = kagglehub.dataset_download("federalreserve/interest-rates")

# Load CSV
fomc = pd.read_csv(f"{path}/index.csv")

print(fomc.head())


Using Colab cache for faster access to the 'interest-rates' dataset.
   Year  Month  Day  Federal Funds Target Rate  Federal Funds Upper Target  \
0  1954      7    1                        NaN                         NaN   
1  1954      8    1                        NaN                         NaN   
2  1954      9    1                        NaN                         NaN   
3  1954     10    1                        NaN                         NaN   
4  1954     11    1                        NaN                         NaN   

   Federal Funds Lower Target  Effective Federal Funds Rate  \
0                         NaN                          0.80   
1                         NaN                          1.22   
2                         NaN                          1.06   
3                         NaN                          0.85   
4                         NaN                          0.83   

   Real GDP (Percent Change)  Unemployment Rate  Inflation Rate  
0                